In [22]:
import pandas as pd

In [81]:
df = pd.read_csv("outcomes.csv", header=None)
df.columns = ["strategy 1", "strategy 2", "score 1", "score 2", "count"]

In [82]:
df.describe()

,score 1,score 2,count
count,794731.000000,794731.000000,794731.000000
mean,2.210296,2.274771,5.546594
std,0.749463,0.879763,41.762043
min,0.000000,0.000000,1.000000
25%,1.720000,1.595000,1.000000
50%,2.180000,2.320000,1.000000
75%,2.665000,2.920000,3.000000
max,5.000000,5.000000,1000.000000


Creating a dictionary `f` that maps a name of strategy tuples to the mean utility for that matchup:


In [83]:
def r(*scores):
    """
    Calculate the fitness ratio r based on the utility of two strategies.
    
    Returns the tuple of fitness ratios (first element is inverse of second).
    """
    
    if scores[1] == 0:
        # The second strategy scores 0
        # Returns inf, 0
        r = float('inf')
        return r, 1 / r
    elif scores[0] == 0:
        # The first strategy scores 0
        # Returns 0, inf
        invr = float('inf')
        return 1 / invr, invr
    else:
        r = scores[0] / scores[1]
        return r, 1/r
        
fitness = {pair: r(f["score 1"].mean(), f["score 2"].mean()) 
           for pair, f in df.groupby(["strategy 1", "strategy 2"])}

Writing that to file.

In [84]:
with open("fitness.csv", "a") as outfile:
    for key, value in fitness.items():
        outfile.write("{}, {}, {}, {}\n".format(*key, *value))